In [ ]:
from flask import Flask,request,send_file,render_template , request 
import re
import scipy, base64
import scipy.io.wavfile as wavf

from flask_ngrok import run_with_ngrok
import os, pydub

from google.colab import drive
drive.mount('/content/gdrive')

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/", methods=['POST'])
def trump():
  id = 'id'
  # print(request.json)
  paragraph = request.json["prompt"]
  # sentences = paragraph.split(", ")
  # for text in sentences:
  #   print(text)

  sentences = re.split('; |, |\. |\n',paragraph)
  # sentences = paragraph.split(" ")
  in_fpath = Path("trump11.wav")
  reprocessed_wav = encoder.preprocess_wav(in_fpath)
  original_wav, sampling_rate = librosa.load(in_fpath)
  preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
  embed = encoder.embed_utterance(preprocessed_wav)
  generated_audio = None
  for sent in sentences:
    print(sent)
    audio = synth(sent,embed)
    display(Audio(audio, rate=synthesizer.sample_rate))
    if generated_audio is None:
      generated_audio = audio
    else:
      generated_audio = np.append(generated_audio,audio)
      # generated_audio = np.pad(generated_audio, (0, audio), mode="constant")
      # generated_audio = generated_audio + audio
  display(Audio(generated_audio, rate=synthesizer.sample_rate))
  out_f = 'out.wav'
  fs = synthesizer.sample_rate
  wavf.write(out_f, fs, generated_audio)

  sound = pydub.AudioSegment.from_wav("out.wav")
  sound.export("out.mp3", format="mp3")
  
  return send_file(
            f"out.wav", 
            mimetype="audio/wav", 
            as_attachment=True)
  
  # return {
  #     # "data": str(base64.b64encode(generated_audio))
  #     "data": generated_audio.tolist()
  # }

if __name__ == '__main__':
  app.run()

# !$env:FLASK_APP="app.ipynb"
!flask run